In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers
!pip install pytorch-nlp

     |████████████████████████████████| 3.1 MB 14.9 MB/s 
     |████████████████████████████████| 895 kB 81.0 MB/s 
     |████████████████████████████████| 3.3 MB 81.7 MB/s 
     |████████████████████████████████| 61 kB 650 kB/s 
     |████████████████████████████████| 596 kB 80.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 90 kB 7.3 MB/s 


In [4]:
import sys
import os

os.chdir('/content/drive/MyDrive/NLPProject/bias' )
sys.path.append('/content/drive/MyDrive/NLPProject')
sys.path.append('/content/drive/MyDrive/NLPProject/bias/')
os.environ['OMP_NUM_THREADS'] = "1"

import argparse
import pandas as pd
import pickle
from model.generator import TransformerDataset, transformer_collate
from model.bertmodel import MyBertModel
from model.emobertmodel import emoMyBertModel
from model.emo_Attention2_bertmodel import emoAtt2MyBertModel
from model.lstmmodel import LSTMModel
import torch
from parameters import BERT_MODEL_PATH, CLAIM_ONLY, CLAIM_AND_EVIDENCE, EVIDENCE_ONLY, DEVICE, INPUT_TYPE_ORDER
from transformers import AdamW
import numpy as np
from utils.utils import print_message, clean_str, preprocess
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter
from torchnlp.word_to_vector import GloVe
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
#from hypopt import GridSearch
from model_selection import GridSearch
from tqdm import tqdm


# load the right dataset for different configurations, if preprocessing
# is set, it will preprocess the dataset
def load_data(dataset, step = 'none'):
    #path = "../../multi_fc_publicdata/" + dataset + "/"

    path = "../multi_fc_publicdata/" + dataset + "/"
    print(f'load data function: step = {step}')
    if step == 'formal' or step == 'informal':
      print('**** load data: formal or informal *****') 
      main_data = pd.read_csv(path + dataset + "_" + step + ".tsv", sep="\t", header=None)
      snippets_data = pd.read_csv(path + dataset + "_" + step + "_snippets.tsv", sep="\t", header=None)    
    elif step == 'none' or step == 'EMO_LEXI' or step == 'EMO_INT' or step == 'EMO_ATT2_LEXI' or step == 'EMO_ATT2_INT':
      print('****load data: none-preprocess, EMO_LEXI, EMO_INT, EMO_ATT2_LEXI, EMO_ATT2_INT *****') 
      main_data = pd.read_csv(path + dataset + ".tsv", sep="\t", header=None)
      snippets_data = pd.read_csv(path + dataset + "_snippets.tsv", sep="\t", header=None)    
    elif step == 'neutralized': 
      print('****load data: neutralized *****') 
      main_data = pd.read_csv(path + dataset + "_" + step + ".tsv", sep="\t", header=None)     
      snippets_data = pd.read_csv(path + dataset + "_snippets.tsv", sep="\t", header=None)    
    else: 
      print('****load data: preprocess *****') 
      main_data = pd.read_csv(path + dataset + ".tsv", sep="\t", header=None)
      snippets_data = pd.read_csv(path + dataset + "_snippets.tsv", sep="\t", header=None)

      for index, row in main_data.iterrows():
        main_data[1][index] = preprocess(row[1], step)     
      
          
      for index, row in snippets_data.iterrows():
        snippets_data[1][index] = preprocess(row[1], step)
        snippets_data[2][index] = preprocess(row[2], step)
        snippets_data[3][index] = preprocess(row[3], step)
        snippets_data[4][index] = preprocess(row[4], step)
        snippets_data[5][index] = preprocess(row[5], step)
        snippets_data[6][index] = preprocess(row[6], step)
        snippets_data[7][index] = preprocess(row[7], step)
        snippets_data[8][index] = preprocess(row[8], step)
        snippets_data[9][index] = preprocess(row[9], step)
        snippets_data[10][index] = preprocess(row[10], step)
    
    label_order = pickle.load(open(path + dataset + "_labels.pkl", "rb"))
    splits = pickle.load(open(path + dataset + "_index_split.pkl", "rb"))

    return main_data, snippets_data, label_order, splits

# Generating dataset loader for the claims and snippet data.
def make_generators(main_data, snippets_data, label_order, splits, params, dataset_generator=TransformerDataset, other_dataset=False):
    generators = []

    all_labels = main_data.values[:,2]
    counter = Counter(all_labels)
    ss = ""
    for c in label_order:
        ss = ss + ", " + str(c) + " (" + str(np.around(counter[c]/len(all_labels) * 100,1)) + "\%)"
        #print(c, np.around(counter[c]/len(all_labels) * 100,1), "%", counter[c])
    print("len", len(all_labels), ss)

    for isplit, split in enumerate(splits):
        # print(f'isplit {isplit}')
        sub_main_data = main_data.values[split]
        # print(f'len sub_main_data: {len(sub_main_data)}')
        
        sub_snippets_data = snippets_data.values[split]
        # print(f'len sub_snippets_data: {len(sub_snippets_data)}')

        

        tmp = dataset_generator(sub_main_data, sub_snippets_data, label_order)
        if isplit == 0:
            generator = torch.utils.data.DataLoader(tmp, **params[0])
        else:
            generator = torch.utils.data.DataLoader(tmp, **params[1])

        generators.append(generator)

        # print(sub_main_data)
        # print(sub_snippets_data)
        # print(f'tmp: \n {tmp[0]}')
        # gen0 = next(iter(generator))
        # print(f'gen0: \n {gen0}')


    # make class weights
    labels = main_data.values[splits[0]][:,2]
    labels = np.array([label_order.index(v) for v in labels])


    if not other_dataset:
        label_weights = torch.tensor(compute_class_weight("balanced", classes=np.arange(len(label_order)), y=labels).astype(np.float32))
    else:
        label_weights = None

    return generators[0], generators[1], generators[2], label_weights

# evaluate the f1micro and f1 macro scores
def evaluate(generator, model, other_from=None, ignore_snippet=None):
    all_labels = []
    all_predictions = []

    all_claimIDs = []
    all_logits = []

    for vals in generator:
        claimIDs, claims, labels, snippets = vals[0], vals[1], vals[2], vals[3]

        if ignore_snippet is not None:
            for i in range(len(snippets)):
                snippets[i][ignore_snippet] = "filler"

        all_labels += labels
        logits = model(claims, snippets)

        predictions = torch.argmax(logits, 1).cpu().numpy()

        if other_from == "pomt": # other data is pomt, and model is trained on snes
            # this case is fine
            pass
        elif other_from == "snes": # other data is snes, and model is trained on pomt
            # in this case both "pants on fire!" and "false" should be considered as false
            predictions[predictions == 0] = 1 # 0 is "pants on fire!" and 1 is "false" for pomt.

        all_predictions += predictions.tolist()

        all_claimIDs += claimIDs
        all_logits += logits.cpu().numpy().tolist()

    f1_micro = f1_score(all_labels, all_predictions, average="micro")
    f1_macro = f1_score(all_labels, all_predictions, average="macro")

    return f1_micro, f1_macro, all_claimIDs, all_logits, all_labels, all_predictions

def train_step(optimizer, vals, model, criterion):
    optimizer.zero_grad()

    claimIDs, claims, labels, snippets = vals[0], vals[1], torch.tensor(vals[2]).to(DEVICE), vals[3]

    logits = model(claims, snippets)
    loss = criterion(logits, labels)

    loss.backward()
    optimizer.step()

    return loss


# get embedding matric according to Glove('840B')
def get_embedding_matrix(generators, dataset, min_occurrence=1):
    savename = "preprocessed/" + dataset + "_glove.pkl"
    if os.path.exists(savename):
        tmp = pickle.load(open(savename, "rb"))
        glove_embedding_matrix = tmp[0]
        word2idx = tmp[1]
        idx2word = tmp[2]
        return glove_embedding_matrix, word2idx, idx2word

    glove_vectors = GloVe('840B')
    all_claims = []
    all_snippets = []
    for gen in generators:
        for vals in gen:
            claims = vals[1]
            claims = [clean_str(v) for v in claims]
            snippets = vals[3]
            snippets = [clean_str(item) for sublist in snippets for item in sublist]

            all_claims += claims
            all_snippets += snippets

    all_words = [word for v in all_claims+all_snippets for word in v.split(" ")]
    counter = Counter(all_words)
    all_words = set(all_words)
    all_words = list(set([word for word in all_words if counter[word] > min_occurrence]))
    word2idx = {word: i+2 for i, word in enumerate(all_words)} # reserve 0 for potential mask and 1 for unk token
    idx2word = {word2idx[key]: key for key in word2idx}

    num_words = len(idx2word)

    glove_embedding_matrix = np.random.random((num_words+2, 300)) - 0.5
    missed = 0
    for word in word2idx:
        if word in glove_vectors:
            glove_embedding_matrix[word2idx[word]] = glove_vectors[word]
        else:
            missed += 1

    pickle.dump([glove_embedding_matrix, word2idx, idx2word], open(savename, "wb"))
    return glove_embedding_matrix, word2idx, idx2word

def train_model(model, criterion, optimizer, train_generator, val_generator, test_generator, args, other_generator, savename):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    print("model parameters", params)

    num_epochs = 0
    patience_counter = 0
    # patience_max = 1
    patience_max = 8

    best_f1 = -np.inf
    while (True):
        train_losses = []

        model.train()
        for ivals, vals in enumerate(train_generator):
            loss = train_step(optimizer, vals, model, criterion)
            train_losses.append(loss.item())

        num_epochs += 1
        print_message("TRAIN loss", np.mean(train_losses), num_epochs)

        if num_epochs % args.eval_per_epoch == 0:
            model.eval()
            with torch.no_grad():
                val_f1micro, val_f1macro, val_claimIDs, val_logits, val_labels, val_predictions = evaluate(val_generator, model)
                print_message("VALIDATION F1micro, F1macro, loss:", val_f1micro, val_f1macro, len(val_claimIDs))

            if val_f1macro > best_f1:
                with torch.no_grad():
                    test_f1micro, test_f1macro, test_claimIDs, test_logits, test_labels, test_predictions = evaluate(test_generator, model)
                    print_message("TEST F1micro, F1macro, loss:", test_f1micro, test_f1macro, len(test_claimIDs))

                    other_test_f1micro, other_test_f1macro, other_test_claimIDs, other_test_logits, other_test_labels, other_test_predictions = evaluate(other_generator, model, other_from="snes" if args.dataset == "pomt" else "pomt")
                    print_message("OTHER-TEST F1micro, F1macro, loss:", other_test_f1micro, other_test_f1macro, len(other_test_claimIDs))

                    test_remove_top_bottom = []
                    test_remove_bottom_top = []
                    other_test_remove_top_bottom = []
                    other_test_remove_bottom_top = []
                    ten = np.arange(10)
                    if args.inputtype != "CLAIM_ONLY":
                        for i in tqdm(range(10)):
                            top_is = ten[:(i+1)]
                            bottom_is = ten[-(i+1):]
                            test_remove_top_bottom.append( evaluate(test_generator, model, ignore_snippet=top_is) )
                            test_remove_bottom_top.append( evaluate(test_generator, model, ignore_snippet=bottom_is) )
                            other_test_remove_top_bottom.append(evaluate(other_generator, model, other_from="snes" if args.dataset == "pomt" else "pomt", ignore_snippet=top_is))
                            other_test_remove_bottom_top.append(evaluate(other_generator, model, other_from="snes" if args.dataset == "pomt" else "pomt", ignore_snippet=bottom_is))

                        print_message([np.around(v[1], 4) for v in test_remove_top_bottom])
                        print_message([np.around(v[1], 4) for v in test_remove_bottom_top])
                        print_message([np.around(v[1], 4) for v in other_test_remove_top_bottom])
                        print_message([np.around(v[1], 4) for v in other_test_remove_bottom_top])

                patience_counter = 0
                best_f1 = val_f1macro
                val_store = [val_f1micro, val_f1macro, val_claimIDs, val_logits, val_labels, val_predictions]
                test_store = [test_f1micro, test_f1macro, test_claimIDs, test_logits, test_labels, test_predictions, test_remove_top_bottom, test_remove_bottom_top]
                other_test_store = [other_test_f1micro, other_test_f1macro, other_test_claimIDs, other_test_logits, other_test_labels, other_test_predictions, other_test_remove_top_bottom, other_test_remove_bottom_top]
                misc_store = [args]
                total_store = [val_store, test_store, other_test_store, misc_store]
            else:
                patience_counter += 1

            print_message("PATIENCE", patience_counter, "/", patience_max)

            if patience_counter >= patience_max:
                pickle.dump(total_store, open(savename, "wb"))
                break

# run the bert model
def run_bert(args, train_generator, val_generator, test_generator, label_weights, inputtype, label_order, savename, other_generator, step):
    print(f'***run_bert*** with inputtype {args.inputtype}')

    if step == 'EMO_INT' or step == 'EMO_LEXI':      
      model = emoMyBertModel.from_pretrained(BERT_MODEL_PATH, labelnum=len(label_order), input_type=inputtype, emocred_type = step)
    elif step == 'EMO_ATT2_INT' or step == 'EMO_ATT2_LEXI':      
      model = emoAtt2MyBertModel.from_pretrained(BERT_MODEL_PATH, labelnum=len(label_order), input_type=inputtype, emocred_type = step)
    else:
      model = MyBertModel.from_pretrained(BERT_MODEL_PATH, labelnum=len(label_order), input_type=inputtype)
    model.to(DEVICE)

    criterion = torch.nn.CrossEntropyLoss(weight=label_weights.to(DEVICE))
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=args.lr, eps=1e-8)
    optimizer.zero_grad()

    train_model(model, criterion, optimizer, train_generator, val_generator, test_generator, args, other_generator, savename)

# run lstm model
def run_lstm(args, train_generator, val_generator, test_generator, label_weights, inputtype, label_order, savename, other_generator):
    print(f'***run_lstm*** with inputtype {args.inputtype}')
    glove_embedding_matrix, word2idx, idx2word = get_embedding_matrix([train_generator, val_generator, test_generator, other_generator], args.dataset)

    model = LSTMModel(args.lstm_hidden_dim, args.lstm_layers, args.lstm_dropout, len(label_order), word2idx, glove_embedding_matrix, input_type=inputtype)
    model.to(DEVICE)

    criterion = torch.nn.CrossEntropyLoss(weight=label_weights.to(DEVICE))
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=args.lr, eps=1e-8)
    optimizer.zero_grad()

    train_model(model, criterion, optimizer, train_generator, val_generator, test_generator, args, other_generator, savename)

def filter_snippet_for_bow(generator, ignore_snippet, inputtype):
    samples = []
    for vals in generator:
        claims = vals[1]
        labels = vals[2]
        snippets = vals[3]

        for i in range(len(snippets)):
            snippets[i][ignore_snippet] = "filler"

        for i in range(len(claims)):
            if inputtype == CLAIM_AND_EVIDENCE:
                sample = clean_str(claims[i]) + " ".join([clean_str(v) for v in snippets[i]])
            elif inputtype == CLAIM_ONLY:
                sample = clean_str(claims[i])
            elif inputtype == EVIDENCE_ONLY:
                sample = " ".join([clean_str(v) for v in snippets[i]])
            else:
                raise Exception("Unknown type", inputtype)
            samples.append(sample)
    return samples

def get_bows_labels(generators, dataset, inputtype):
    all_samples = []
    all_labels = []

    for gen in generators:
        gen_samples = []
        gen_labels = []
        for vals in gen:
            claims = vals[1]
            labels = vals[2]
            snippets = vals[3]

            for i in range(len(claims)):
                if inputtype == CLAIM_AND_EVIDENCE:
                    sample = clean_str(claims[i]) + " ".join([clean_str(v) for v in snippets[i]])
                elif inputtype == CLAIM_ONLY:
                    sample = clean_str(claims[i])
                elif inputtype == EVIDENCE_ONLY:
                    sample = " ".join([clean_str(v) for v in snippets[i]])
                else:
                    raise Exception("Unknown type", inputtype)
                gen_samples.append(sample)
                gen_labels.append(labels[i])

        all_samples.append(gen_samples)
        all_labels.append(gen_labels)

    test_remove_top_bottom = []
    test_remove_bottom_top = []
    other_test_remove_top_bottom = []
    other_test_remove_bottom_top = []
    ten = np.arange(10)
    for i in tqdm(range(10)):
        top_is = ten[:(i + 1)]
        bottom_is = ten[-(i + 1):]
        test_remove_top_bottom.append( filter_snippet_for_bow(generators[-2], top_is, inputtype) )
        test_remove_bottom_top.append( filter_snippet_for_bow(generators[-2], bottom_is, inputtype) )
        other_test_remove_top_bottom.append( filter_snippet_for_bow(generators[-1], top_is, inputtype) )
        other_test_remove_bottom_top.append( filter_snippet_for_bow(generators[-1], bottom_is, inputtype) )

    vectorizer = TfidfVectorizer(min_df=2)
    vectorizer.fit([item for sublist in all_samples for item in sublist])

    bows = [vectorizer.transform(all_samples[i]) for i in range(len(all_samples))]

    test_remove_top_bottom = [vectorizer.transform(test_remove_top_bottom[i]) for i in range(len(test_remove_top_bottom))]
    test_remove_bottom_top = [vectorizer.transform(test_remove_bottom_top[i]) for i in range(len(test_remove_bottom_top))]
    other_test_remove_top_bottom = [vectorizer.transform(other_test_remove_top_bottom[i]) for i in range(len(other_test_remove_top_bottom))]
    other_test_remove_bottom_top = [vectorizer.transform(other_test_remove_bottom_top[i]) for i in range(len(other_test_remove_bottom_top))]

    return bows, all_labels, test_remove_top_bottom, test_remove_bottom_top, other_test_remove_top_bottom, other_test_remove_bottom_top

# random forest model
def run_bow(args, train_generator, val_generator, test_generator, label_weights, inputtype, label_order, savename, other_test_generator):
    # print(f'train_generator0 :\n {next(iter(train_generator))}')
    print(f'***run_bow*** with inputtype {args.inputtype}')

    bows, labels, test_remove_top_bottom, test_remove_bottom_top, other_test_remove_top_bottom, other_test_remove_bottom_top = get_bows_labels([train_generator, val_generator, test_generator, other_test_generator], args.dataset, inputtype)

    train_bow, val_bow, test_bow, other_test_bow = bows[0], bows[1], bows[2], bows[3]
    train_labels, val_labels, test_labels, other_test_labels = labels[0], labels[1], labels[2], labels[3]

    label_weights = label_weights.numpy()
    weights = {i: label_weights[i] for i in range(len(label_weights))}

    # print(f'****** run bow train_bow \n {train_bow}')
    # print('*********')

    param_grid = [
        {'n_estimators': [100, 500, 1000], 'min_samples_leaf': [1, 3, 5, 10], 'min_samples_split': [2, 5, 10]}
    ]

    opt = GridSearch(model=RandomForestClassifier(n_jobs=5, class_weight=weights), param_grid=param_grid, parallelize=False)

    
    opt.fit(train_bow, train_labels, val_bow, val_labels, scoring="f1_macro")

    def rf_eval(model, bow, labels, other_from=None):
        preds = model.predict(bow)

        if other_from == "pomt": # other data is pomt, and model is trained on snes
            # this case is fine
            pass
        elif other_from == "snes": # other data is snes, and model is trained on pomt
            # in this case both "pants on fire!" and "false" should be considered as false
            preds[preds == 0] = 1 # 0 is "pants on fire!" and 1 is "false" for pomt.

        f1_macro = f1_score(labels, preds, average="macro")
        f1_micro = f1_score(labels, preds, average="micro")
        return f1_micro, f1_macro, labels, preds

    # val_store = [val_f1micro, val_f1macro, val_claimIDs, val_logits, val_labels, val_predictions]
    # test_store = [test_f1micro, test_f1macro, test_claimIDs, test_logits, test_labels, test_predictions,test_remove_top_bottom, test_remove_bottom_top]
    # other_test_store = [other_test_f1micro, other_test_f1macro, other_test_claimIDs, other_test_logits,
    #                     other_test_labels, other_test_predictions, other_test_remove_top_bottom,
    #                     other_test_remove_bottom_top]
    #misc_store = [args]


    val_store = rf_eval(opt, val_bow, val_labels)
    test_store = list(rf_eval(opt, test_bow, test_labels)) + [[rf_eval(opt, test_remove_top_bottom[i], test_labels) for i in range(10)],
                                                       [rf_eval(opt, test_remove_bottom_top[i], test_labels) for i in range(10)]]
    other_test_store = list(rf_eval(opt, other_test_bow, other_test_labels, other_from="snes" if args.dataset == "pomt" else "pomt")) + [[rf_eval(opt, other_test_remove_top_bottom[i], other_test_labels, other_from="snes" if args.dataset == "pomt" else "pomt") for i in range(10)],
                                                       [rf_eval(opt, other_test_remove_bottom_top[i], other_test_labels, other_from="snes" if args.dataset == "pomt" else "pomt") for i in range(10)]]
    misc_store = [opt.get_best_params()]
    total_store = [val_store, test_store, other_test_store, misc_store]

    print_message("VALIDATION", val_store[0], val_store[1])
    print_message("TEST", test_store[0], test_store[1])
    print_message("OTHER-TEST", other_test_store[0], other_test_store[1])

    print_message([np.around(v[1], 4) for v in test_store[-2]])
    print_message([np.around(v[1], 4) for v in test_store[-1]])
    print_message([np.around(v[1], 4) for v in other_test_store[-2]])
    print_message([np.around(v[1], 4) for v in other_test_store[-1]])
    print(misc_store)

    pickle.dump(total_store, open(savename, "wb"))

def filter_websites(snippets_data):
    bad_websites = ["factcheck.org", "politifact.com", "snopes.com", "fullfact.org", "factscan.ca"]
    ids = snippets_data.values[:, 0]
    remove_count = 0
    for i, id in enumerate(ids):
        with open("../../multi_fc_publicdata/snippets/" + id, "r", encoding="utf-8") as f:
            lines = f.readlines()

        links = [line.strip().split("\t")[-1] for line in lines]
        remove = [False for _ in range(10)]
        for j in range(len(links)):
            remove[j] = any([bad in links[j] for bad in bad_websites])
        remove = remove[:10]  # 1 data sample has 11 links by mistake in the dataset
        snippets_data.iloc[i, [False] + remove] = "filler"

        remove_count += np.sum(remove)
    print_message("REMOVE COUNT", remove_count)
    return snippets_data



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Error loading SnowballStemmer: Package 'SnowballStemmer'
[nltk_data]     not found in index
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# don't include formal and informal in the steps, need to have separate cell due to we don't have formal and informal data for politifact

# the cell will run all the training and evaluation according to the configurations, 
# and generate the results and store under the results folder
import gc

gc.collect()
class vars():
    def __init__(self, mode, inputtype, dataset):
        if mode == "bow":
            self.dataset = dataset
            self.inputtype = inputtype
            self.filter_websites = 0
            self.model = "bow"
            self.batchsize = 2
            self.eval_per_epoch = 1
            self.lr = 0.0001
        elif mode == 'lstm':
            self.dataset = dataset
            self.inputtype = inputtype
            self.filter_websites = 0
            self.model = "lstm"
            self.batchsize = 16
            self.eval_per_epoch = 1
            self.lr = 0.0001
            self.lstm_hidden_dim = 128
            self.lstm_layers = 2
            self.lstm_dropout = 0.1
        elif mode == 'bert':
            self.dataset = dataset
            self.inputtype = inputtype
            self.filter_websites = 0
            self.model = "bert"
            if self.dataset == "snes":
              self.batchsize = 6
            elif self.dataset == "pomt":
              self.batchsize = 4
            self.eval_per_epoch = 1
            self.lr = 0.000003            

filepath = 'sorted.uk.word.unigrams'  
word_freq = {}  
count = 0
with open(filepath, encoding= 'utf-8') as f:
    for line in f:
        line = line.rstrip()
        if line:
            x = line.split('\t')
            #print(x)
            #print(key, val)
            #print(str(x[1]))
            word_freq[x[1]] = str(x[0])
        count +=1
        if count > 100000:
            break

steps = [['none'], ['pos'], ['stop'], ['pos', 'stop'], ['EMO_INT'], ['EMO_ATT2_INT'], ['EMO_LEXI'], ['EMO_ATT2_LEXI'], ['neg'], ['stem'], ['pos', 'neg'], ['pos', 'neg', 'stop'], ['all']]
modes = ['bert']
datasets = ['snes', 'pomt']
inputtypes = ['CLAIM_ONLY', 'CLAIM_AND_EVIDENCE', 'EVIDENCE_ONLY']
# inputtypes = ['CLAIM_ONLY']



for step in steps:
  for mode in modes:    
    for dataset in datasets:
      for inputtype in inputtypes:       
        print('***********************************')
        stepstr = "-".join([s for s in step])                
        args = vars(mode, inputtype, dataset)

        if args.filter_websites > 0.5:
            savename = "results/" + "-".join([str(v) for v in [args.filter_websites, args.model, args.dataset, args.inputtype, stepstr, args.lr, args.batchsize]])
        else:
            savename = "results/" + "-".join([str(v) for v in [args.model, args.dataset, args.inputtype, stepstr, args.lr, args.batchsize]])

        if args.model == "lstm":
            savename += "-" + "-".join([str(v) for v in [args.lstm_hidden_dim, args.lstm_layers, stepstr, args.lstm_dropout]])
        savename += ".pkl"
        print(args.inputtype, "-", args.dataset, "-", stepstr)
        inputtype = INPUT_TYPE_ORDER.index(args.inputtype)
        main_data, snippets_data, label_order, splits = load_data(args.dataset, stepstr)

        if args.filter_websites > 0.5:
            snippets_data = filter_websites(snippets_data)

        params = {"batch_size": args.batchsize, "shuffle": True, "num_workers": 1, "collate_fn": transformer_collate, "persistent_workers": True, "prefetch_factor":5}
        eval_params = {"batch_size": args.batchsize, "shuffle": False, "num_workers": 1, "collate_fn": transformer_collate, "persistent_workers": True, "prefetch_factor":5}

        train_generator, val_generator, test_generator, label_weights = make_generators(main_data, snippets_data, label_order, splits, [params, eval_params])

        if stepstr == 'formal' or stepstr == 'informal':
          if args.dataset == "snes":
              main_data, snippets_data, _, splits = load_data("pomt", 'none')
              if args.filter_websites > 0.5:
                  snippets_data = filter_websites(snippets_data)
              main_data.iloc[main_data.iloc[:, 2] == "pants on fire!", 2] = "false"
              main_data.iloc[main_data.iloc[:, 2] == "half-true", 2] = "mixture"
              _, _, other_test_generator, _ = make_generators(main_data, snippets_data, label_order, splits, [params, eval_params], other_dataset=True)
          else:
              main_data, snippets_data, _, splits = load_data("snes", 'stepstr')
              if args.filter_websites > 0.5:
                  snippets_data = filter_websites(snippets_data)
              main_data.iloc[main_data.iloc[:, 2] == "mixture", 2] = "half-true"
              _, _, other_test_generator, _ = make_generators(main_data, snippets_data, label_order, splits, [params, eval_params], other_dataset=True)
        else:
          if args.dataset == "snes":
              main_data, snippets_data, _, splits = load_data("pomt", stepstr)
              if args.filter_websites > 0.5:
                  snippets_data = filter_websites(snippets_data)
              main_data.iloc[main_data.iloc[:, 2] == "pants on fire!", 2] = "false"
              main_data.iloc[main_data.iloc[:, 2] == "half-true", 2] = "mixture"
              _, _, other_test_generator, _ = make_generators(main_data, snippets_data, label_order, splits, [params, eval_params], other_dataset=True)
          else:
              main_data, snippets_data, _, splits = load_data("snes", stepstr)
              if args.filter_websites > 0.5:
                  snippets_data = filter_websites(snippets_data)
              main_data.iloc[main_data.iloc[:, 2] == "mixture", 2] = "half-true"
              _, _, other_test_generator, _ = make_generators(main_data, snippets_data, label_order, splits, [params, eval_params], other_dataset=True)


        if args.model == "bert":
            run_bert(args, train_generator, val_generator, test_generator, label_weights, inputtype, label_order, savename, other_test_generator, stepstr)
        elif args.model == "lstm":
            run_lstm(args, train_generator, val_generator, test_generator, label_weights, inputtype, label_order, savename, other_test_generator)
        elif args.model == "bow":
            # print("run bow")
            run_bow(args, train_generator, val_generator, test_generator, label_weights, inputtype, label_order, savename, other_test_generator)

        gc.collect()




***********************************
CLAIM_ONLY - snes - none
load data function: step = none
****load data: none-preprocess, EMO_LEXI, EMO_INT, EMO_ATT2_LEXI, EMO_ATT2_INT *****
len 5069 , false (64.3\%), mostly false (7.5\%), mixture (12.3\%), mostly true (2.8\%), true (13.0\%)
load data function: step = none
****load data: none-preprocess, EMO_LEXI, EMO_INT, EMO_ATT2_LEXI, EMO_ATT2_INT *****
len 13581 , false (29.7\%), mostly false (17.0\%), mixture (19.8\%), mostly true (18.8\%), true (14.8\%)
***run_bert*** with inputtype CLAIM_ONLY


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

...

model parameters 109486085
[Dec 07, 06:14:57] TRAIN loss 1.5616625064128153 1
[Dec 07, 06:14:59] VALIDATION F1micro, F1macro, loss: 0.5266272189349113 0.21374930304326964 507
[Dec 07, 06:15:01] TEST F1micro, F1macro, loss: 0.5443786982248521 0.22592877473634593 1014
[Dec 07, 06:15:08] OTHER-TEST F1micro, F1macro, loss: 0.2602134707397865 0.15994827610981216 2717
[Dec 07, 06:15:08] PATIENCE 0 / 1
[Dec 07, 06:15:39] TRAIN loss 1.5099589542762653 2
[Dec 07, 06:15:41] VALIDATION F1micro, F1macro, loss: 0.5581854043392505 0.25861395789629754 507
[Dec 07, 06:15:43] TEST F1micro, F1macro, loss: 0.5611439842209073 0.2669040471583006 1014
[Dec 07, 06:15:50] OTHER-TEST F1micro, F1macro, loss: 0.24880382775119617 0.1988867741977735 2717
[Dec 07, 06:15:50] PATIENCE 0 / 1
[Dec 07, 06:16:21] TRAIN loss 1.4229062467410758 3
[Dec 07, 06:16:23] VALIDATION F1micro, F1macro, loss: 0.5818540433925049 0.2909908782485241 507
[Dec 07, 06:16:25] TEST F1micro, F1macro, loss: 0.5650887573964497 0.27090401843304

Some weights of the model checkpoint at bert-base-uncased were not used when initializing MyBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing MyBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MyBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MyBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['predictor.weight', 'pred

model parameters 109486854
[Dec 07, 06:22:42] TRAIN loss 1.7527585546660434 1
[Dec 07, 06:22:47] VALIDATION F1micro, F1macro, loss: 0.23932253313696614 0.23741218098793568 1358
[Dec 07, 06:22:58] TEST F1micro, F1macro, loss: 0.23923444976076555 0.23615350059872983 2717
[Dec 07, 06:23:02] OTHER-TEST F1micro, F1macro, loss: 0.5867850098619329 0.21139776231979565 1014
[Dec 07, 06:23:02] PATIENCE 0 / 1
[Dec 07, 06:25:06] TRAIN loss 1.65608539045183 2
[Dec 07, 06:25:11] VALIDATION F1micro, F1macro, loss: 0.2496318114874816 0.2472208380103199 1358
[Dec 07, 06:25:22] TEST F1micro, F1macro, loss: 0.24806772175193229 0.2432515429277964 2717
[Dec 07, 06:25:26] OTHER-TEST F1micro, F1macro, loss: 0.5946745562130178 0.20551429737418947 1014
[Dec 07, 06:25:26] PATIENCE 0 / 1
[Dec 07, 06:27:31] TRAIN loss 1.5557329946824068 3
[Dec 07, 06:27:36] VALIDATION F1micro, F1macro, loss: 0.25773195876288657 0.25388749692355383 1358
[Dec 07, 06:27:47] TEST F1micro, F1macro, loss: 0.2539565697460434 0.253761730

Some weights of the model checkpoint at bert-base-uncased were not used when initializing MyBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing MyBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MyBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MyBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['predictor.weight', 'pred

model parameters 109486085
[Dec 07, 06:43:39] TRAIN loss 1.569862731707257 1
[Dec 07, 06:43:40] VALIDATION F1micro, F1macro, loss: 0.46942800788954636 0.2189597148317089 507
[Dec 07, 06:43:43] TEST F1micro, F1macro, loss: 0.5177514792899408 0.26229640050494096 1014
[Dec 07, 06:43:50] OTHER-TEST F1micro, F1macro, loss: 0.25653294074346705 0.1856654523506282 2717
[Dec 07, 06:43:50] PATIENCE 0 / 1
[Dec 07, 06:44:20] TRAIN loss 1.4994780793786049 2
[Dec 07, 06:44:21] VALIDATION F1micro, F1macro, loss: 0.4792899408284023 0.21896659670119437 507
[Dec 07, 06:44:24] TEST F1micro, F1macro, loss: 0.5019723865877712 0.2518601883122589 1014
[Dec 07, 06:44:31] OTHER-TEST F1micro, F1macro, loss: 0.2491718807508281 0.15743322663909357 2717
[Dec 07, 06:44:31] PATIENCE 0 / 1
[Dec 07, 06:45:02] TRAIN loss 1.4296688549220562 3
[Dec 07, 06:45:03] VALIDATION F1micro, F1macro, loss: 0.4970414201183432 0.2429788097409506 507
[Dec 07, 06:45:05] TEST F1micro, F1macro, loss: 0.5256410256410257 0.271064928347418

In [ ]:
# this cell is for formal and informal in the steps, need to have separate cell due to we don't have formal and informal data for politifact
# the cell will run all the training and evaluation according to the configurations, 
# and generate the results and store under the results folder

import gc

gc.collect()
class vars():
    def __init__(self, mode, inputtype, dataset):
        if mode == "bow":
            self.dataset = dataset
            self.inputtype = inputtype
            self.filter_websites = 0
            self.model = "bow"
            self.batchsize = 2
            self.eval_per_epoch = 1
            self.lr = 0.0001
        elif mode == 'lstm':
            self.dataset = dataset
            self.inputtype = inputtype
            self.filter_websites = 0
            self.model = "lstm"
            self.batchsize = 16
            self.eval_per_epoch = 1
            self.lr = 0.0001
            self.lstm_hidden_dim = 128
            self.lstm_layers = 2
            self.lstm_dropout = 0.1
        elif mode == 'bert':
            self.dataset = dataset
            self.inputtype = inputtype
            self.filter_websites = 0
            self.model = "bert"
            if self.dataset == "snes":
              self.batchsize = 6
            elif self.dataset == "pomt":
              self.batchsize = 4
            self.eval_per_epoch = 1
            self.lr = 0.000003            

filepath = 'sorted.uk.word.unigrams'  
word_freq = {}  
count = 0
with open(filepath, encoding= 'utf-8') as f:
    for line in f:
        line = line.rstrip()
        if line:
            x = line.split('\t')
            #print(x)
            #print(key, val)
            #print(str(x[1]))
            word_freq[x[1]] = str(x[0])
        count +=1
        if count > 100000:
            break

# only for formal and informal in the steps, need to have separate cell due to we don't have formal and informal data for politifact
steps = [['formal'], ['informal']]
modes = ['bert']
datasets = ['snes']
inputtypes = ['CLAIM_AND_EVIDENCE', 'EVIDENCE_ONLY', 'CLAIM_ONLY']


for step in steps:
  for mode in modes:    
    for dataset in datasets:
      for inputtype in inputtypes:       
        print('***********************************')
        stepstr = "-".join([s for s in step])                
        args = vars(mode, inputtype, dataset)

        if args.filter_websites > 0.5:
            savename = "results/" + "-".join([str(v) for v in [args.filter_websites, args.model, args.dataset, args.inputtype, stepstr, args.lr, args.batchsize]])
        else:
            savename = "results/" + "-".join([str(v) for v in [args.model, args.dataset, args.inputtype, stepstr, args.lr, args.batchsize]])

        if args.model == "lstm":
            savename += "-" + "-".join([str(v) for v in [args.lstm_hidden_dim, args.lstm_layers, stepstr, args.lstm_dropout]])
        savename += ".pkl"
        print(args.inputtype, "-", args.dataset, "-", stepstr)
        inputtype = INPUT_TYPE_ORDER.index(args.inputtype)
        main_data, snippets_data, label_order, splits = load_data(args.dataset, stepstr)

        if args.filter_websites > 0.5:
            snippets_data = filter_websites(snippets_data)

        params = {"batch_size": args.batchsize, "shuffle": True, "num_workers": 1, "collate_fn": transformer_collate, "persistent_workers": True, "prefetch_factor":5}
        eval_params = {"batch_size": args.batchsize, "shuffle": False, "num_workers": 1, "collate_fn": transformer_collate, "persistent_workers": True, "prefetch_factor":5}

        train_generator, val_generator, test_generator, label_weights = make_generators(main_data, snippets_data, label_order, splits, [params, eval_params])

        if stepstr == 'formal' or stepstr == 'informal':
          if args.dataset == "snes":
              main_data, snippets_data, _, splits = load_data("pomt", 'none')
              if args.filter_websites > 0.5:
                  snippets_data = filter_websites(snippets_data)
              main_data.iloc[main_data.iloc[:, 2] == "pants on fire!", 2] = "false"
              main_data.iloc[main_data.iloc[:, 2] == "half-true", 2] = "mixture"
              _, _, other_test_generator, _ = make_generators(main_data, snippets_data, label_order, splits, [params, eval_params], other_dataset=True)
          else:
              main_data, snippets_data, _, splits = load_data("snes", 'stepstr')
              if args.filter_websites > 0.5:
                  snippets_data = filter_websites(snippets_data)
              main_data.iloc[main_data.iloc[:, 2] == "mixture", 2] = "half-true"
              _, _, other_test_generator, _ = make_generators(main_data, snippets_data, label_order, splits, [params, eval_params], other_dataset=True)
        else:
          if args.dataset == "snes":
              main_data, snippets_data, _, splits = load_data("pomt", stepstr)
              if args.filter_websites > 0.5:
                  snippets_data = filter_websites(snippets_data)
              main_data.iloc[main_data.iloc[:, 2] == "pants on fire!", 2] = "false"
              main_data.iloc[main_data.iloc[:, 2] == "half-true", 2] = "mixture"
              _, _, other_test_generator, _ = make_generators(main_data, snippets_data, label_order, splits, [params, eval_params], other_dataset=True)
          else:
              main_data, snippets_data, _, splits = load_data("snes", stepstr)
              if args.filter_websites > 0.5:
                  snippets_data = filter_websites(snippets_data)
              main_data.iloc[main_data.iloc[:, 2] == "mixture", 2] = "half-true"
              _, _, other_test_generator, _ = make_generators(main_data, snippets_data, label_order, splits, [params, eval_params], other_dataset=True)


        if args.model == "bert":
            run_bert(args, train_generator, val_generator, test_generator, label_weights, inputtype, label_order, savename, other_test_generator, stepstr)
        elif args.model == "lstm":
            run_lstm(args, train_generator, val_generator, test_generator, label_weights, inputtype, label_order, savename, other_test_generator)
        elif args.model == "bow":
            # print("run bow")
            run_bow(args, train_generator, val_generator, test_generator, label_weights, inputtype, label_order, savename, other_test_generator)

        gc.collect()


